# Order Brushing Solution (Shopee Code League 2020)

This solution has score **0.89933**

In [29]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

Read dataset and explore some attributes

In [30]:
d1 = pd.read_csv("./data/order_brush_order.csv", header=0, parse_dates=[3])
d1.dtypes

orderid                int64
shopid                 int64
userid                 int64
event_time    datetime64[ns]
dtype: object

In [31]:
row, column = d1.shape
print("Row: ", row)

Row:  222750


In [32]:
d1.describe().T

,count,mean,std,min,25%,50%,75%,max
orderid,222750.0,3.130027e+13,1.222774e+11,3.107520e+13,3.120360e+13,3.130561e+13,3.140604e+13,3.150720e+13
shopid,222750.0,9.433117e+07,5.695790e+07,1.000900e+04,4.980267e+07,9.033636e+07,1.475053e+08,2.154352e+08
userid,222750.0,9.802880e+07,6.839048e+07,1.000700e+04,3.508127e+07,9.309625e+07,1.590612e+08,2.155262e+08


In [33]:
d1.head(10)

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
5,31119725718155,173811070,116451780,2019-12-27 12:22:05
6,31151322178251,107921853,166741763,2019-12-27 21:08:43
7,31079856153738,178400128,61272835,2019-12-27 01:17:37
8,31118426867571,147941492,10986763,2019-12-27 12:00:27
9,31133458226149,164933170,135957741,2019-12-27 16:10:59


Retrieve brushing `orderid` and all relevant `orderid`s.

**Warning**: the following cell may run more than 5 minutes.

In [46]:
delta = pd.Timedelta(hours=1)
order_list = set() # store other rows that relevant to a brushing rows
shop_query = dict() # to store query for shopid to quickly access

i = 0
while i < row:
    # if i % 10000 == 0:
    #     print(i)

    r = d1.iloc[i]
    end_time = r.event_time + delta
    previous_time = r.event_time - delta
    
    q = shop_query[r.shopid] if r.shopid in shop_query else d1[d1.shopid == r.shopid]
    shop_query[r.shopid] = q

    q_n = q[q.event_time.between(r.event_time, end_time)]
    q_p = q[q.event_time.between(previous_time, r.event_time)]

    con_rate_p = len(q_p)/q_p.userid.nunique()
    con_rate_n = len(q_n)/q_n.userid.nunique()

    if con_rate_p >= 3:
        # order_brushing.append(r.orderid)
        order_list.update(list(q_p.orderid.unique()))
    if con_rate_n >= 3:
        order_list.update(list(q_n.orderid.unique()))

    i+=1


Query data set

In [47]:
d2 = d1[d1.orderid.isin(order_list)]

In [48]:
print(d2.shape)
d2_row, d2_column = d2.shape

(1134, 4)


Define a function to find all `userid` with highest user proportion.

In [49]:
def find_max(userid_list, user_p_list):
    max_value = max(user_p_list)
    maxs_index = []
    for i, value in enumerate(user_p_list):
        if value == max_value:
            maxs_index.append(i)
    max_user = [ userid_list[i] for i in maxs_index]
    return max_value, set(max_user)

Define a dictionary to store `shopid` with `userid` and maximum user proportion for that `userid`s

In [50]:
shop_list = dict()

def add_to_list(shopid, max_tuples):
    if shopid not in shop_list:
        shop_list[shopid] = max_tuples
        return
    
    max_value, max_user = shop_list[shopid]
    max_v, max_u = max_tuples
    if max_v > max_value:
        shop_list[shopid] = max_tuples
    elif max_v == max_value:
        shop_list[shopid] = (max_value, max_user.union(max_u))


Process in brushing order, find `userid` for each case and then put into shop_list

In [51]:
d2_uniq_shop = d2.shopid.unique()
for shopid in d2_uniq_shop:
    q = d2[d2.shopid == shopid]

    user_list = q.userid.unique()
    sum_order = len(q)
    user_proportion = []
    for userid in user_list:
        user_proportion.append(len(q[q.userid == userid])/sum_order)

    max_value, max_user = find_max(user_list,user_proportion)
    add_to_list(shopid, (max_value, max_user))

In [52]:
len(d2_uniq_shop)

308

Get unique `shopid`

In [53]:
unique_shopid = d1.shopid.unique()

Process `userid` for each `shopid`

In [54]:
userid_shopid = []
for shopid in unique_shopid:
    userid = "0"
    if shopid in shop_list:
        max_value, max_user = shop_list[shopid]
        userid = "&".join([str(u) for u in sorted(list(max_user))])
    userid_shopid.append(userid)

Save submission

In [55]:
sms = pd.DataFrame({
    "shopid": unique_shopid,
    "userid": userid_shopid
})

In [56]:
sms[sms.userid != "0"]

,shopid,userid
34,1175477,122277324
70,66861410,213502289
72,8715449,9753706
140,58543771,61893096
141,156883302,76102350&188025647
...,...,...
12754,78837196,78903959
12954,189544563,799445
13958,64369518,172591713
14302,83234229,181942741


In [57]:
sms.to_csv("./data/submission.csv", index=False)